### Import das dependências

In [13]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### Importar os datasets

In [14]:
data_path = '../data/'

df_consumos = pd.read_csv(f"{data_path}3-consumos-faturados-por-municipio-ultimos-10-anos.csv", delimiter=';')
df_mobilidade_eletrica = pd.read_csv(f"{data_path}9-plr-mobilidade-eletrica.csv", delimiter=';')
df_diagramas_carga = pd.read_csv(f"{data_path}22-diagrama-de-carga-por-instalacao.csv", delimiter=';')
df_caracterizacao_contrato_ativo = pd.read_csv(f"{data_path}20-caracterizacao-pes-contrato-ativo.csv", delimiter=';')
df_contadores_energia = pd.read_csv(f"{data_path}21-contadores-de-energia.csv", delimiter=';', low_memory=False)
df_ordens_servico = pd.read_csv(f"{data_path}15-ordens-de-servico.csv", delimiter=';')


### 1. Tratar coleções uma a um
Colocar os datatypes, agrupar antes de juntar as várias coleções numa

### 1.1 - Consumos mensais por concelho

In [15]:
df_consumos.describe()


,Ano,Mês,Energia Ativa (kWh),CodDistrito,CodDistritoConcelho
count,173912.000000,173912.000000,1.739120e+05,173912.000000,173912.000000
mean,2021.947818,6.621274,8.338499e+05,9.640209,972.305246
std,0.957736,3.611179,2.651379e+06,5.494382,550.497447
min,2020.000000,1.000000,2.370000e+02,1.000000,101.000000
25%,2021.000000,3.000000,6.657747e+04,4.000000,408.000000
50%,2022.000000,7.000000,1.863577e+05,10.000000,1006.000000
75%,2023.000000,10.000000,5.916892e+05,14.000000,1416.000000
max,2024.000000,12.000000,7.548685e+07,18.000000,1824.000000


In [16]:
# Visualizar os tipos de dados
print("\nInformações sobre os tipos de dados:")
print(df_consumos.info())



Informações sobre os tipos de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Ano                           173912 non-null  int64  
 1   Mês                           173912 non-null  int64  
 2   Data                          173912 non-null  object 
 3   Distrito                      173912 non-null  object 
 4   Concelho                      173912 non-null  object 
 5   Freguesia                     173912 non-null  object 
 6   Nível de Tensão               173912 non-null  object 
 7   Energia Ativa (kWh)           173912 non-null  float64
 8   CodDistrito                   173912 non-null  int64  
 9   CodDistritoConcelho           173912 non-null  int64  
 10  CodDistritoConcelhoFreguesia  173320 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 14.6+ MB
None


In [17]:
# Converter as colunas para o tipo categórico
df_consumos['Ano'] = df_consumos['Ano'].astype('category')
df_consumos['Mês'] = df_consumos['Mês'].astype('category')
df_consumos['Nível de Tensão'] = df_consumos['Nível de Tensão'].astype('category')

# Verificar as mudanças
print(df_consumos.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   Ano                           173912 non-null  category
 1   Mês                           173912 non-null  category
 2   Data                          173912 non-null  object  
 3   Distrito                      173912 non-null  object  
 4   Concelho                      173912 non-null  object  
 5   Freguesia                     173912 non-null  object  
 6   Nível de Tensão               173912 non-null  category
 7   Energia Ativa (kWh)           173912 non-null  float64 
 8   CodDistrito                   173912 non-null  int64   
 9   CodDistritoConcelho           173912 non-null  int64   
 10  CodDistritoConcelhoFreguesia  173320 non-null  object  
dtypes: category(3), float64(1), int64(2), object(5)
memory usage: 11.1+ MB
None


In [18]:
# Iterar sobre todas as colunas do DataFrame
for column in df_consumos.columns:
    # Verificar se a coluna é do tipo categórico
    if df_consumos[column].dtype.name == 'category':
        # Imprimir todas as categorias únicas da coluna
        print(f"Todas as categorias únicas da coluna '{column}':")
        print(df_consumos[column].cat.categories)


Todas as categorias únicas da coluna 'Ano':
Index([2020, 2021, 2022, 2023, 2024], dtype='int64')
Todas as categorias únicas da coluna 'Mês':
Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64')
Todas as categorias únicas da coluna 'Nível de Tensão':
Index(['Baixa Tensão', 'Muito Alta, Alta e Média Tensões'], dtype='object')


### Agrupar por concelho

In [22]:
# Agrupar os dados por ano, mês, concelho e tipo de tensão, somando os valores de 'Energia Ativa (kWh)'
df_consumos_sem_freguesia = df_consumos.drop(columns=['Distrito','Freguesia', 'Data', 'CodDistrito', 'CodDistritoConcelho', 'CodDistritoConcelhoFreguesia'])
# Exibir o DataFrame resultante
df_consumos_sem_freguesia.head(15)


,Ano,Mês,Concelho,Nível de Tensão,Energia Ativa (kWh)
0,2021,4,Ponte de Lima,Baixa Tensão,104321.726
1,2021,5,Amares,"Muito Alta, Alta e Média Tensões",45123.071
2,2021,5,Vizela,Baixa Tensão,1669013.735
3,2021,5,Campo Maior,"Muito Alta, Alta e Média Tensões",450138.731
4,2021,5,Sardoal,Baixa Tensão,44856.247
5,2021,6,Vieira do Minho,"Muito Alta, Alta e Média Tensões",171359.782
6,2021,6,Bombarral,Baixa Tensão,482473.048
7,2021,6,Felgueiras,Baixa Tensão,101726.041
8,2021,6,Póvoa de Varzim,Baixa Tensão,763835.428
9,2021,6,Ponte de Lima,Baixa Tensão,34802.676


In [23]:
# Para a coleção de Consumos Mensais por Concelho
df_consumos_odivelas = df_consumos_sem_freguesia[df_consumos_sem_freguesia['Concelho'] == 'Odivelas']
df_consumos_odivelas

,Ano,Mês,Concelho,Nível de Tensão,Energia Ativa (kWh)
849,2022,2,Odivelas,Baixa Tensão,2166366.580
1376,2021,5,Odivelas,"Muito Alta, Alta e Média Tensões",2027665.179
2299,2023,6,Odivelas,"Muito Alta, Alta e Média Tensões",822406.614
2308,2023,9,Odivelas,Baixa Tensão,7907209.255
3091,2022,9,Odivelas,Baixa Tensão,1950241.104
...,...,...,...,...,...
169978,2023,9,Odivelas,Baixa Tensão,2061230.742
170790,2022,5,Odivelas,Baixa Tensão,4384314.692
170861,2022,11,Odivelas,Baixa Tensão,4232794.958
171098,2021,4,Odivelas,"Muito Alta, Alta e Média Tensões",423540.637


In [21]:
### 1.1 - Consumos mensais por concelho